<div style="background-color:orange; color:white; text-align:center; padding:10px; border-radius:5px;">
    <h1>Détectez des faux billets avec R ou Python</h1>
</div>

# OBJECTIF DE CE NOTEBOOK

Ce notebook est un détecteur de billets, il analyse à partir d'un modele choisi et d'un fichier csv quels sont les billets vrais des billets faux

Mission : Nous sommes consultant *Data Analyst*, Votre entreprise vient de décrocher un contrat avec l’Organisation nationale de lutte contre le faux-monnayage (ONCFM) et elle souhaite vous y envoyer en tant que senior data analyst pour cette mission.Pour aller plus vite dans cette lutte, nous souhaitons mettre à disposition des équipes une application de machine learning. Celle-ci leur permettra, après avoir scanné des billets (longueur, hauteur, largeur, etc.), de faire une prédiction sur la nature des billets (vrai billet ou faux billet).

<div style="background-color:orange; color:white; text-align:center; padding:10px; border-radius:5px;">
<h2 style="margin: auto; padding: 20px; color:#fff; ">Importation des librairies, chargements et Réorganisation de l'algorithme</h2>
</div>

In [37]:
import pandas as pd
import numpy as np
from pathlib import Path
import joblib

<div style="background-color: RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color:#fff; ">Importation du modèle</h3>
</div>

In [40]:
# Chargement du modèle complet
model_folder = Path(r'C:\Users\Public\algorithme') 
model_file = 'Detections_Billets.joblib' 
model_path = model_folder / model_file

if not model_folder.is_dir():
    raise FileNotFoundError(f'Le dossier {model_folder} n\'existe pas.')
elif not model_path.is_file():
    raise FileNotFoundError(f'Le fichier {model_file} est manquant.')
else:
    std_scale, pca, regression = joblib.load(model_path)
    print('Modèle chargé avec succès.')

Modèle chargé avec succès.


<div style="background-color: RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color:#fff; ">Choix du fichier à analyser</h3>
</div>

In [43]:
# Chargement du fichier CSV de test
data_folder = Path(r'C:\Users\Public\algorithme\dataframe')
csv_files = [x for x in data_folder.glob('*.csv') if x.is_file()]

if not csv_files:
    raise FileNotFoundError(f'Aucun fichier CSV trouvé dans {data_folder}')
    
df = pd.read_csv(csv_files[0])

<div style="background-color: RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color:#fff; ">Vérification des données et détection</h3>
</div>

In [46]:
# Vérification des colonnes attendues
features = ['diagonal', 'height_left', 'height_right', 'margin_low', 'margin_up', 'length', 'id']
missing_features = set(features) - set(df.columns)
if missing_features:
    raise Exception(f'Variables manquantes : {missing_features}')
if df.isna().any().any():
    raise Exception(f'Valeurs manquantes dans : {df.columns[df.isna().any()].to_list()}')

# Séparation des colonnes explicatives
X = df[[
    'diagonal', 'height_left', 'height_right',
    'margin_low', 'margin_up', 'length'
]]
ids = df['id']

# Standardisation avec conservation des noms de colonnes
X_scaled = std_scale.transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Prédictions
proba_genuine = regression.predict_proba(X_scaled)[:, 1]
is_genuine = proba_genuine > 0.5

<div style="background-color: RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color:#fff; ">Affichage des résultats</h3>
</div>

In [49]:
# Création d'un DataFrame de résultats sans modifier df
results = df.copy()
results['proba_genuine'] = proba_genuine
results['is_genuine'] = is_genuine

# Réorganisation des colonnes
results = results[[
    'diagonal', 'height_left', 'height_right',
    'margin_low', 'margin_up', 'length',
    'id', 'proba_genuine', 'is_genuine'
]]

# Affichage des résultats individuels
print("Résultats :\n")
for is_real, id_ in zip(results["is_genuine"], results["id"]):
    if is_real:
        print(f"Le billet ref. {id_} est un vrai billet.")
    else:
        print(f"Le billet ref. {id_} est un faux billet.")

# Optionnel : affichage du tableau final
print("\nDétails de la détection :\n")
print(results)


Résultats :

Le billet ref. B_1 est un vrai billet.
Le billet ref. B_2 est un faux billet.
Le billet ref. B_3 est un vrai billet.
Le billet ref. B_4 est un faux billet.
Le billet ref. B_5 est un faux billet.

Détails de la détection :

   diagonal  height_left  height_right  margin_low  margin_up  length   id  \
0    172.09       103.95        103.73        4.39       3.09  113.19  B_1   
1    171.52       104.17        104.03        5.27       3.16  111.82  B_2   
2    171.78       103.80        103.75        3.81       3.24  113.39  B_3   
3    172.02       104.08        103.99        5.57       3.30  111.10  B_4   
4    171.79       104.34        104.37        5.00       3.07  111.87  B_5   

   proba_genuine  is_genuine  
0       0.995398        True  
1       0.005816       False  
2       0.999363        True  
3       0.000079       False  
4       0.010885       False  
